# Обработка текстов

Вам дан файл `ru.conversations.txt`

В нем содержатся диалоги (разделены пустой строкой).

Вам необходимо:

### Разбить текст на диалоги

In [1]:
with open('ru.conversations.txt', 'r', encoding='utf-8') as f:
    text = []
    for line in f:
        text.append(line)

In [2]:
text[:10]

['- Что,  Мансур,  не жарко теперь тебе?\n',
 '- Спрашиваю, не жарко ему теперь?\n',
 '\n',
 '- Какой полк?\n',
 '- Тысяча тридцать четвертый.\n',
 '- Вези дальше. Тут тысяча двадцать шестой.\n',
 '\n',
 '- Какая санрота?\n',
 '- Тысяча тридцать шестая.\n',
 '- Значит, наша! Принимай тяжелораненого!\n']

In [3]:
dialogs = []
dialog = ''
for item in text:
    if item != '\n':
        dialog += item
    else:
        dialogs.append(dialog)
        dialog = ''
print(dialogs[:5])

['- Что,  Мансур,  не жарко теперь тебе?\n- Спрашиваю, не жарко ему теперь?\n', '- Какой полк?\n- Тысяча тридцать четвертый.\n- Вези дальше. Тут тысяча двадцать шестой.\n', '- Какая санрота?\n- Тысяча тридцать шестая.\n- Значит, наша! Принимай тяжелораненого!\n', '- Сколько фрицев в котле?\n- Тысяч сорок.\n', '- Чем же он отравился?\n- А вон, видишь, что-то из тех бутылей выпил.\n']


### Разбить диалоги на токены

In [4]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize

In [5]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pro10\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
for i,item in enumerate(dialogs):
    dialogs[i] = word_tokenize(item)

### Предобработать токены
Удалить стоп-слова и спецсимволы, лемматизировать или стеммировать слова и т.д.

In [7]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

In [8]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pro10\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
tokenizer = RegexpTokenizer(r'\w+')
for i,item in enumerate(dialogs):
    dialogs[i] = tokenizer.tokenize(str(dialogs[i]))

In [10]:
clean_dialogs = dialogs
for i, dialog in enumerate(dialogs):
    clean_dialog = dialog
    for word in dialog:
        if word.lower() in stopwords.words('russian'):
            clean_dialog.remove(word)
    clean_dialogs[i] = clean_dialog

In [11]:
clean_dialogs[:5]

[['Мансур', 'жарко', 'тебе', 'Спрашиваю', 'жарко', 'теперь'],
 ['полк',
  'Тысяча',
  'тридцать',
  'четвертый',
  'Вези',
  'дальше',
  'тысяча',
  'двадцать',
  'шестой'],
 ['санрота',
  'Тысяча',
  'тридцать',
  'шестая',
  'Значит',
  'наша',
  'Принимай',
  'тяжелораненого'],
 ['Сколько', 'фрицев', 'котле', 'Тысяч', 'сорок'],
 ['же', 'отравился', 'вон', 'видишь', 'то', 'тех', 'бутылей', 'выпил']]

In [12]:
from nltk.stem import SnowballStemmer
rus_stemmer = SnowballStemmer('russian')

In [13]:
%%timeit
rus_stemmer.stem('котле')

97 µs ± 16.8 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [14]:
stem_dialogs = clean_dialogs
for i, dialog in enumerate(clean_dialogs):
    stem_dialog = dialog
    for j, word in enumerate(dialog):
        stem_dialog[j] = rus_stemmer.stem(word)
    stem_dialogs[i] = stem_dialog

### Перевести каждый из диалогов в векторное представление

С помощью TF-IDF представить каждый из диалогов в векторном формате

In [15]:
from sklearn.feature_extraction.text import CountVectorizer

In [16]:
' '.join(map(str, stem_dialogs[1]))

'полк тысяч тридца четверт вез дальш тысяч двадца шест'

In [19]:
vectorizer = CountVectorizer(ngram_range=(1, 2))
vectorizer.fit([' '.join(map(str, stem_dialogs[1]))])
vectors_arr_0 = vectorizer.transform([' '.join(map(str, stem_dialogs[1]))]).toarray()
vectors_arr_0, ' '.join(map(str, stem_dialogs[1]))

(array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1]], dtype=int64),
 'полк тысяч тридца четверт вез дальш тысяч двадца шест')

In [20]:
vectorizer.vocabulary_

{'полк': 6,
 'тысяч': 10,
 'тридца': 8,
 'четверт': 13,
 'вез': 0,
 'дальш': 2,
 'двадца': 4,
 'шест': 15,
 'полк тысяч': 7,
 'тысяч тридца': 12,
 'тридца четверт': 9,
 'четверт вез': 14,
 'вез дальш': 1,
 'дальш тысяч': 3,
 'тысяч двадца': 11,
 'двадца шест': 5}

In [25]:
import sklearn.feature_extraction.text as txt

In [32]:
stem_dialogs[1]

['полк',
 'тысяч',
 'тридца',
 'четверт',
 'вез',
 'дальш',
 'тысяч',
 'двадца',
 'шест']

In [34]:
tfvec.vocabulary_

{'полк': 3,
 'тысяч': 5,
 'тридца': 4,
 'четверт': 6,
 'вез': 0,
 'дальш': 1,
 'двадца': 2,
 'шест': 7}

In [38]:
tfvec = txt.TfidfVectorizer(ngram_range=(1, 2))
two_sent_embed = tfvec.fit_transform([' '.join(map(str, stem_dialogs[1])), ' '.join(map(str, stem_dialogs[2]))]).toarray()

print(two_sent_embed)

[[0.25364289 0.25364289 0.25364289 0.25364289 0.25364289 0.25364289
  0.         0.         0.         0.         0.25364289 0.25364289
  0.         0.         0.         0.         0.180469   0.25364289
  0.         0.36093801 0.25364289 0.180469   0.         0.25364289
  0.25364289 0.180469   0.        ]
 [0.         0.         0.         0.         0.         0.
  0.27708406 0.27708406 0.27708406 0.27708406 0.         0.
  0.27708406 0.27708406 0.27708406 0.27708406 0.19714759 0.
  0.27708406 0.19714759 0.         0.19714759 0.27708406 0.
  0.         0.19714759 0.27708406]]


In [40]:
from scipy.spatial.distance import euclidean

In [41]:
euclidean(two_sent_embed[0], two_sent_embed[1])

1.2822674108617793

### Найти самые близкие по содержанию диалоги, исходя из векторного представления